# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
import sklearn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!pip install scikit-learn==0.20

    100% |████████████████████████████████| 5.3MB 5.4MB/s eta 0:00:01   4% |█▌                              | 245kB 11.9MB/s eta 0:00:01    33% |██████████▊                     | 1.8MB 16.2MB/s eta 0:00:01    74% |███████████████████████▊        | 4.0MB 23.1MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.19.1
    Uninstalling scikit-learn-0.19.1:
      Successfully uninstalled scikit-learn-0.19.1


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql ('SELECT * FROM MessagesTable', engine)
df.head()
X = df['message']
y = df.iloc[:,4:]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
categories_df = df[df.columns[4:]]
cat_count = categories_df.sum().sort_values(ascending=False)
top5_cat = cat_count[:5]
top5_cat_names = list(top5_cat.index)
top5_cat_names

['related', 'aid_related', 'weather_related', 'direct_report', 'request']

In [13]:
y.dtypes

related                   int64
request                   int64
offer                     int64
aid_related               int64
medical_help              int64
medical_products          int64
search_and_rescue         int64
security                  int64
military                  int64
child_alone               int64
water                     int64
food                      int64
shelter                   int64
clothing                  int64
money                     int64
missing_people            int64
refugees                  int64
death                     int64
other_aid                 int64
infrastructure_related    int64
transport                 int64
buildings                 int64
electricity               int64
tools                     int64
hospitals                 int64
shops                     int64
aid_centers               int64
other_infrastructure      int64
weather_related           int64
floods                    int64
storm                     int64
fire    

In [7]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    """
    Tokenization function to process the text data. As taught in class, the process consists of the following processes:
    Normalize, Tokenize, Stemming, Lemmatize and extract stop words. Returns a list of tokenized words.
    
    input: Text from the messages column
    
    output: tokenized text
    
    """
    # Normalize
    text = re.sub('\W', ' ', text.lower())
    
    stop_words = stopwords.words('english')
    
    # Tokenize, Stemming and Lemmatizing
    tokens = word_tokenize(text)
    words_stemmed = [PorterStemmer().stem(tok) for tok in tokens]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words_stemmed if w not in stop_words]
    
    return lemmed

In [11]:
# Testinng if the function really works
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
estimators = [('vect', CountVectorizer(tokenizer=tokenize)),('tfidf', TfidfTransformer()),('clf', MultiOutputClassifier(RandomForestClassifier()))]
pipeline = Pipeline(estimators)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)
y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
report = pd.DataFrame()
for index, col in enumerate(y_test.columns):
    print (col, classification_report(y_test.iloc[:,index], y_pred[:,index], target_names=y_test.columns))

related                         precision    recall  f1-score   support

               related       0.62      0.47      0.54      1557
               request       0.84      0.91      0.88      4941
                 offer       0.40      0.43      0.41        47

           avg / total       0.79      0.80      0.79      6545

request                         precision    recall  f1-score   support

               related       0.88      0.98      0.93      5393
               request       0.79      0.40      0.53      1152

           avg / total       0.87      0.88      0.86      6545

offer                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      6516
               request       0.00      0.00      0.00        29

           avg / total       0.99      1.00      0.99      6545

aid_related                         precision    recall  f1-score   support

               related       0.75      0.86      0.80      3

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  >>> y_pred = [0, 1, 0, 0, 0, 1]
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  >>> y_pred = [0, 1, 0, 0, 0, 1]
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  >>> y_pred = [0, 1, 0, 0, 0, 1]


In [19]:
report.head()

,0,1,micro avg,macro avg,weighted avg
f1-score,0.909753,0.445415,0.844767,0.677584,0.819865
precision,0.856355,0.722124,0.844767,0.789239,0.830370
recall,0.970254,0.322021,0.844767,0.646137,0.844767
support,5278.000000,1267.000000,6545.000000,6545.000000,6545.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__max_df': (0.75,1.0),
             'clf__estimator__n_estimators':[10,20],
             'clf__estimator__min_samples_split': [2,5]
             }

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,y_train)
y_pred_tuned = cv.predict(X_test)

In [ ]:
# Show the accuracy, precision, and recall of the optimized model
for index, col in enumerate(y_test.columns):
    print (col, classification_report(y_test.iloc[:,index], y_pred[:,index], target_names=y_test.columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# I'll use the StartingVerbExtractor taught in class and test the model. Also I made some modifications to include the cases of message with no information
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
            

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
# Creating a new pipeline using the custom transformer and FeatureUnion to combine it with TF-IDF
pipeline = Pipeline([
        ('features', FeatureUnion ([
            
            ('text_pipeline', Pipeline ([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
                                        ])),      
            ('starting_verb', StartingVerbExtractor ())   
                                    ])),        
        
        ('clf', MultiOutputClassifier (RandomForestClassifier ()))
        ])

In [ ]:
pipeline.fit(X_train,y_train)
y_pred_model = pipeline.predict(X_test)

In [ ]:
# Show the accuracy, precision, and recall of the new model
for index, col in enumerate(y_test.columns):
    print (col, classification_report(y_test.iloc[:,index], y_pred[:,index], target_names=y_test.columns))

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('ml_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.